In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import random
import numpy as np
import torch
import math

In [3]:
from PIL import Image, ImageOps
from argparse import ArgumentParser

from torch.optim import SGD, Adam, lr_scheduler
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, CenterCrop, Normalize, Resize, Pad
from torchvision.transforms import ToTensor, ToPILImage

In [4]:
import importlib
from iouEval import iouEval, getColorEntry

In [5]:
from dataset import lanes
from transform import Relabel, Relabel_enc, ToLabel, Colorize
from visualize import Dashboard
from shutil import copyfile

In [6]:
NUM_CHANNELS = 3
NUM_CLASSES = 5 #pascal=22, cityscapes=20

color_transform = Colorize(NUM_CLASSES)
image_transform = ToPILImage()


In [7]:
#Augmentations - different function implemented to perform random augments on both image and target
class MyCoTransform(object):
    def __init__(self, enc, augment=True, height=512):
        self.enc=enc
        self.augment = augment
        self.height = height
        pass
    def __call__(self, input, target):
        # do something to both images
        input =  Resize(self.height, Image.BILINEAR)(input)
        targets1 = Resize((self.height, 904), Image.NEAREST)(target[0])
        targets2 = Resize((self.height, 904), Image.NEAREST)(target[1])
        targets3 = Resize((self.height, 904), Image.NEAREST)(target[2])
        targets4 = Resize((self.height, 904), Image.NEAREST)(target[3])
        targets5 = Resize((self.height, 904), Image.NEAREST)(target[4])

        if(self.augment):
            # Random hflip
            hflip = random.random()
            if (hflip < 0.5):
                input = input.transpose(Image.FLIP_LEFT_RIGHT)
                targets1 = targets1.transpose(Image.FLIP_LEFT_RIGHT)
                targets2 = targets2.transpose(Image.FLIP_LEFT_RIGHT)
                targets3 = targets3.transpose(Image.FLIP_LEFT_RIGHT)
                targets4 = targets4.transpose(Image.FLIP_LEFT_RIGHT)
                targets5 = targets5.transpose(Image.FLIP_LEFT_RIGHT)
            
            #Random translation 0-2 pixels (fill rest with padding
            transX = random.randint(-2, 2) 
            transY = random.randint(-2, 2)

            input = ImageOps.expand(input, border=(transX,transY,0,0), fill=0)
            targets1 = ImageOps.expand(targets1, border=(transX,transY,0,0), fill=255) #pad label filling with 255
            targets2 = ImageOps.expand(targets2, border=(transX,transY,0,0), fill=255) #pad label filling with 255
            targets3 = ImageOps.expand(targets3, border=(transX,transY,0,0), fill=255) #pad label filling with 255
            targets4 = ImageOps.expand(targets4, border=(transX,transY,0,0), fill=255) #pad label filling with 255
            targets5 = ImageOps.expand(targets5, border=(transX,transY,0,0), fill=255) #pad label filling with 255
            input = input.crop((0, 0, input.size[0]-transX, input.size[1]-transY))
            targets1 = targets1.crop((0, 0, targets1.size[0]-transX, targets1.size[1]-transY))
            targets2 = targets2.crop((0, 0, targets2.size[0]-transX, targets2.size[1]-transY))
            targets3 = targets3.crop((0, 0, targets3.size[0]-transX, targets3.size[1]-transY))
            targets4 = targets4.crop((0, 0, targets4.size[0]-transX, targets4.size[1]-transY))
            targets5 = targets5.crop((0, 0, targets5.size[0]-transX, targets5.size[1]-transY))

        input = ToTensor()(input)
        if (self.enc):
            targets1 = Resize(int(self.height/8), Image.NEAREST)(targets1)
            targets2 = Resize(int(self.height/8), Image.NEAREST)(targets2)
            targets3 = Resize(int(self.height/8), Image.NEAREST)(targets3)
            targets4 = Resize(int(self.height/8), Image.NEAREST)(targets4)
            targets5 = Resize(int(self.height/8), Image.NEAREST)(targets5)
        target = np.zeros_like(targets1)
        targets1, targets2, targets3, targets4, targets5, target = ToLabel()(targets1), ToLabel()(targets2), ToLabel()(targets3), ToLabel()(targets4), ToLabel()(targets5), ToLabel()(target)
        return input, Relabel_enc()(targets1, targets2, targets3, targets4, targets5, target)
        
        # else:
        #     targets1 = ToLabel()(targets1)
        #     #targets1 = Relabel(255, 1)(targets1)
        #     targets2 = ToLabel()(targets2)
        #     #targets2 = Relabel(255, 2)(targets2)
        #     targets3 = ToLabel()(targets3)
        #     #targets3 = Relabel(255, 3)(targets3)
        #     targets4 = ToLabel()(targets4)
        #     #targets4 = Relabel(255, 4)(targets4)
        #     targets5 = ToLabel()(targets5)
        #     #targets5 = Relabel(255, 5)(targets5)
            
        #     target = torch.cat((targets1,targets2,targets3,targets4,targets5), 0)

        #     return input, target

In [8]:
class CrossEntropyLoss2d(torch.nn.Module):

    def __init__(self, weight=None):
        super().__init__()

        self.loss = torch.nn.NLLLoss2d(weight)

    def forward(self, outputs, targets):
        return self.loss(torch.nn.functional.log_softmax(outputs, dim=1), targets)

In [9]:
def train(model, enc=False, cuda=True, train_datadir="train_data_2/", test_datadir="test_data_2/", resume=True, num_epochs=250,
         visualize=False, steps_plot=50, port=8888, iouTrain=False, iouVal=True, height=512, num_workers=0, batch_size=4,
         steps_loss=50, epochs_save=0):
    best_acc = 0

    #TODO: calculate weights by processing dataset histogram (now its being set by hand from the torch values)
    #create a loder to run all images and calculate histogram of labels, then create weight array using class balancing

    weight = torch.ones(NUM_CLASSES)
    if (enc):
        weight[0] = 22
        weight[1] = 22
        weight[2] = 23
        weight[3] = 22
    else:
        weight[0] = 22
        weight[1] = 23
        weight[2] = 21
        weight[3] = 24

    weight[4] = 0

    assert os.path.exists(train_datadir), "Error: datadir (dataset directory) could not be loaded"

    co_transform = MyCoTransform(enc, augment=True, height=height)#1024)
    co_transform_val = MyCoTransform(enc, augment=False, height=height)#1024)
    dataset_train = lanes(train_datadir, co_transform)
    dataset_val = lanes(test_datadir, co_transform_val)

    loader = DataLoader(dataset_train, num_workers=num_workers, batch_size=batch_size, shuffle=True)
    loader_val = DataLoader(dataset_val, num_workers=num_workers, batch_size=batch_size, shuffle=False)

    if cuda:
        weight = weight.cuda()
    criterion = CrossEntropyLoss2d(weight)
    print(type(criterion))

    savedir = f'../save/'

    if (enc):
        automated_log_path = savedir + "/automated_log_encoder.txt"
        modeltxtpath = savedir + "/model_encoder.txt"
    else:
        automated_log_path = savedir + "/automated_log.txt"
        modeltxtpath = savedir + "/model.txt"    

    if (not os.path.exists(automated_log_path)):    #dont add first line if it exists 
        with open(automated_log_path, "a") as myfile:
            myfile.write("Epoch\t\tTrain-loss\t\tTest-loss\t\tTrain-IoU\t\tTest-IoU\t\tlearningRate")

    with open(modeltxtpath, "w") as myfile:
        myfile.write(str(model))


    #TODO: reduce memory in first gpu: https://discuss.pytorch.org/t/multi-gpu-training-memory-usage-in-balance/4163/4        #https://github.com/pytorch/pytorch/issues/1893

    #optimizer = Adam(model.parameters(), 5e-4, (0.9, 0.999),  eps=1e-08, weight_decay=2e-4)     ## scheduler 1
    optimizer = Adam(model.parameters(), 5e-4, (0.9, 0.999),  eps=1e-08, weight_decay=1e-4)      ## scheduler 2

    start_epoch = 1
    if resume:
        #Must load weights, optimizer, epoch and best value. 
        if enc:
            filenameCheckpoint = savedir + '/checkpoint_enc.pth.tar'
        else:
            filenameCheckpoint = savedir + '/checkpoint.pth.tar'

        assert os.path.exists(filenameCheckpoint), "Error: resume option was used but checkpoint was not found in folder"
        checkpoint = torch.load(filenameCheckpoint)
        start_epoch = checkpoint['epoch']
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        best_acc = checkpoint['best_acc']
        print("=> Loaded checkpoint at epoch {})".format(checkpoint['epoch']))

    #scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5) # set up scheduler     ## scheduler 1
    lambda1 = lambda epoch: pow((1-((epoch-1)/num_epochs)),0.9)  ## scheduler 2
    scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda1)                             ## scheduler 2

    if visualize and steps_plot > 0:
        board = Dashboard(port)

    for epoch in range(start_epoch, num_epochs+1):
        print("----- TRAINING - EPOCH", epoch, "-----")

        scheduler.step(epoch)    ## scheduler 2

        epoch_loss = []
     
        doIouTrain = iouTrain   
        doIouVal =  iouVal      

        if (doIouTrain):
            iouEvalTrain = iouEval(NUM_CLASSES)

        usedLr = 0
        for param_group in optimizer.param_groups:
            print("LEARNING RATE: ", param_group['lr'])
            usedLr = float(param_group['lr'])

        model.train()

        for step, (images, labels) in enumerate(loader):

            #print (labels.size())
            #print (np.unique(labels.numpy()))
            #print("labels: ", np.unique(labels[0].numpy()))
            #labels = torch.ones(4, 1, 512, 1024).long()
            if cuda:
                images = images.cuda()
                labels = labels.cuda()

            inputs = Variable(images)
            targets = Variable(labels)
            outputs = model(inputs, only_encode=enc)

            #print("targets", np.unique(targets[:, 0].cpu().data.numpy()))

            optimizer.zero_grad()
            loss = criterion(outputs, targets[:, 0])
            loss.backward()
            optimizer.step()
            

            epoch_loss.append(loss.data)

            if (doIouTrain):
                iouEvalTrain.addBatch(outputs.max(1)[1].unsqueeze(1).data, targets.data)    

            #print(outputs.size())
            if visualize and steps_plot > 0 and step % steps_plot == 0:
                image = inputs[0].cpu().data
                #image[0] = image[0] * .229 + .485
                #image[1] = image[1] * .224 + .456
                #image[2] = image[2] * .225 + .406
                #print("output", np.unique(outputs[0].cpu().max(0)[1].data.numpy()))
                board.image(image, f'input (epoch: {epoch}, step: {step})')
                if isinstance(outputs, list):   #merge gpu tensors
                    board.image(color_transform(outputs[0][0].cpu().max(0)[1].data.unsqueeze(0)),
                    f'output (epoch: {epoch}, step: {step})')
                else:
                    board.image(color_transform(outputs[0].cpu().max(0)[1].data.unsqueeze(0)),
                    f'output (epoch: {epoch}, step: {step})')
                board.image(color_transform(targets[0].cpu().data),
                    f'target (epoch: {epoch}, step: {step})')
            if steps_loss > 0 and step % steps_loss == 0:
                average = sum(epoch_loss) / len(epoch_loss)
                print(f'loss: {average:0.4} (epoch: {epoch}, step: {step})')

            
        average_epoch_loss_train = sum(epoch_loss) / len(epoch_loss)
        
        iouTrain = 0
        if (doIouTrain):
            iouTrain, iou_classes = iouEvalTrain.getIoU()
            iouStr = getColorEntry(iouTrain)+'{:0.2f}'.format(iouTrain*100) + '\033[0m'
            print ("EPOCH IoU on TRAIN set: ", iouStr, "%")  

        #Validate on 500 val images after each epoch of training
        print("----- VALIDATING - EPOCH", epoch, "-----")
        model.eval()
        epoch_loss_val = []

        if (doIouVal):
            iouEvalVal = iouEval(NUM_CLASSES)

        with torch.no_grad():
            for step, (images, labels) in enumerate(loader_val):
                if cuda:
                    images = images.cuda()
                    labels = labels.cuda()

                inputs = Variable(images, volatile=True)    #volatile flag makes it free backward or outputs for eval
                targets = Variable(labels, volatile=True)
                outputs = model(inputs, only_encode=enc) 
                
                loss = criterion(outputs, targets[:, 0])
                epoch_loss_val.append(loss.data)


                #Add batch to calculate TP, FP and FN for iou estimation
                if (doIouVal):
                    iouEvalVal.addBatch(outputs.max(1)[1].unsqueeze(1).data, targets.data)

                if visualize and steps_plot > 0 and step % steps_plot == 0:
                    image = inputs[0].cpu().data
                    board.image(image, f'VAL input (epoch: {epoch}, step: {step})')
                    if isinstance(outputs, list):   #merge gpu tensors
                        board.image(color_transform(outputs[0][0].cpu().max(0)[1].data.unsqueeze(0)),
                        f'VAL output (epoch: {epoch}, step: {step})')
                    else:
                        board.image(color_transform(outputs[0].cpu().max(0)[1].data.unsqueeze(0)),
                        f'VAL output (epoch: {epoch}, step: {step})')
                    board.image(color_transform(targets[0].cpu().data),
                        f'VAL target (epoch: {epoch}, step: {step})')
                if steps_loss > 0 and step % steps_loss == 0:
                    average = sum(epoch_loss_val) / len(epoch_loss_val)
                    print(f'VAL loss: {average:0.4} (epoch: {epoch}, step: {step})')
                       

        average_epoch_loss_val = sum(epoch_loss_val) / len(epoch_loss_val)
        #scheduler.step(average_epoch_loss_val, epoch)  ## scheduler 1   # update lr if needed

        iouVal = 0
        if (doIouVal):
            iouVal, iou_classes = iouEvalVal.getIoU()
            iouStr = getColorEntry(iouVal)+'{:0.2f}'.format(iouVal*100) + '\033[0m'
            print ("EPOCH IoU on VAL set: ", iouStr, "%") 
           

        # remember best valIoU and save checkpoint
        if iouVal == 0:
            current_acc = -average_epoch_loss_val
        else:
            current_acc = iouVal 
        is_best = current_acc > best_acc
        best_acc = max(current_acc, best_acc)
        if enc:
            filenameCheckpoint = savedir + '/checkpoint_enc.pth.tar'
            filenameBest = savedir + '/model_best_enc.pth.tar'    
        else:
            filenameCheckpoint = savedir + '/checkpoint.pth.tar'
            filenameBest = savedir + '/model_best.pth.tar'
        save_checkpoint({
            'epoch': epoch + 1,
            'arch': str(model),
            'state_dict': model.state_dict(),
            'best_acc': best_acc,
            'optimizer' : optimizer.state_dict(),
        }, is_best, filenameCheckpoint, filenameBest)

        #SAVE MODEL AFTER EPOCH
        if (enc):
            filename = f'{savedir}/model_encoder-{epoch:03}.pth'
            filenamebest = f'{savedir}/model_encoder_best.pth'
        else:
            filename = f'{savedir}/model-{epoch:03}.pth'
            filenamebest = f'{savedir}/model_best.pth'
        if epochs_save > 0 and step > 0 and step % epochs_save == 0:
            torch.save(model.state_dict(), filename)
            print(f'save: {filename} (epoch: {epoch})')
        if (is_best):
            torch.save(model.state_dict(), filenamebest)
            print(f'save: {filenamebest} (epoch: {epoch})')
            if (not enc):
                with open(savedir + "/best.txt", "w") as myfile:
                    myfile.write("Best epoch is %d, with Val-IoU= %.4f" % (epoch, iouVal))   
            else:
                with open(savedir + "/best_encoder.txt", "w") as myfile:
                    myfile.write("Best epoch is %d, with Val-IoU= %.4f" % (epoch, iouVal))           

        #SAVE TO FILE A ROW WITH THE EPOCH RESULT (train loss, val loss, train IoU, val IoU)
        #Epoch		Train-loss		Test-loss	Train-IoU	Test-IoU		learningRate
        with open(automated_log_path, "a") as myfile:
            myfile.write("\n%d\t\t%.4f\t\t%.4f\t\t%.4f\t\t%.4f\t\t%.8f" % (epoch, average_epoch_loss_train, average_epoch_loss_val, iouTrain, iouVal, usedLr ))
    
    return(model)   #return model (convenience for encoder-decoder training)


In [10]:
def save_checkpoint(state, is_best, filenameCheckpoint, filenameBest):
    torch.save(state, filenameCheckpoint)
    if is_best:
        print ("Saving model as best")
        torch.save(state, filenameBest)

In [11]:
def main(modell, decoder=False, state=False, pretrainedEncoder=False, cuda=True):
    savedir = f'../save/'

    if not os.path.exists(savedir):
        os.makedirs(savedir)

    assert os.path.exists(modell + ".py"), "Error: model definition not found"
    model_file = importlib.import_module(modell)
    model = model_file.Net(NUM_CLASSES)
    copyfile(modell + ".py", savedir + '/' + modell + ".py")
    
    if cuda:
        model = torch.nn.DataParallel(model).cuda()
    
    if state:
        #if args.state is provided then load this state for training
        #Note: this only loads initialized weights. If you want to resume a training use "--resume" option!!
        """
        try:
            model.load_state_dict(torch.load(args.state))
        except AssertionError:
            model.load_state_dict(torch.load(args.state,
                map_location=lambda storage, loc: storage))
        #When model is saved as DataParallel it adds a model. to each key. To remove:
        #state_dict = {k.partition('model.')[2]: v for k,v in state_dict}
        #https://discuss.pytorch.org/t/prefix-parameter-names-in-saved-model-if-trained-by-multi-gpu/494
        """
        def load_my_state_dict(model, state_dict):  #custom function to load model when not all dict keys are there
            own_state = model.state_dict()
            for name, param in state_dict.items():
                if name not in own_state:
                     continue
                own_state[name].copy_(param)
            return model

        #print(torch.load(args.state))
        #model = load_my_state_dict(model, torch.load(state))
        model.load_state_dict(torch.load("../save/model_best.pth"))

    #train(args, model)
    if (not decoder):
        print("========== ENCODER TRAINING ===========")
        model = train(model, True) #Train encoder
    #CAREFUL: for some reason, after training encoder alone, the decoder gets weights=0. 
    #We must reinit decoder weights or reload network passing only encoder in order to train decoder
    print("========== DECODER TRAINING ===========")
    if (not state):
        if pretrainedEncoder:
            print("Loading encoder pretrained in imagenet")
            from erfnet_imagenet import ERFNet as ERFNet_imagenet
            pretrainedEnc = torch.nn.DataParallel(ERFNet_imagenet(1000))
            pretrainedEnc.load_state_dict(torch.load(pretrainedEncoder)['state_dict'])
            pretrainedEnc = next(pretrainedEnc.children()).features.encoder
            if (not cuda):
                pretrainedEnc = pretrainedEnc.cpu()     #because loaded encoder is probably saved in cuda
        else:
            pretrainedEnc = next(model.children()).encoder
        model = model_file.Net(NUM_CLASSES, encoder=pretrainedEnc)  #Add decoder to encoder
        if cuda:
            model = torch.nn.DataParallel(model).cuda()
        #When loading encoder reinitialize weights for decoder because they are set to 0 when training dec
    model = train(model, enc=False)   #Train decoder
    print("========== TRAINING FINISHED ===========")


In [12]:
main(modell="erfnet")

========== ENCODER TRAINING ===========
<class '__main__.CrossEntropyLoss2d'>
=> Loaded checkpoint at epoch 151)
----- TRAINING - EPOCH 151 -----
LEARNING RATE:  0.00021919164527704348
loss: 0.01936 (epoch: 151, step: 0)
loss: 0.02076 (epoch: 151, step: 50)
loss: 0.02097 (epoch: 151, step: 100)
----- VALIDATING - EPOCH 151 -----
VAL loss: 0.0191 (epoch: 151, step: 0)
EPOCH IoU on VAL set:  49.51 %
----- TRAINING - EPOCH 152 -----
LEARNING RATE:  0.00021721793047137816
loss: 0.02747 (epoch: 152, step: 0)
loss: 0.02109 (epoch: 152, step: 50)
loss: 0.02113 (epoch: 152, step: 100)
----- VALIDATING - EPOCH 152 -----
VAL loss: 0.01982 (epoch: 152, step: 0)
EPOCH IoU on VAL set:  50.76 %
----- TRAINING - EPOCH 153 -----
LEARNING RATE:  0.00021524222097152554
loss: 0.0205 (epoch: 153, step: 0)
loss: 0.02068 (epoch: 153, step: 50)
loss: 0.02084 (epoch: 153, step: 100)
----- VALIDATING - EPOCH 153 -----
VAL loss: 0.01746 (epoch: 153, step: 0)
EPOCH IoU on VAL set:  51.23 %
----- TRAINING - EPOCH

loss: 0.01898 (epoch: 178, step: 50)
loss: 0.01906 (epoch: 178, step: 100)
----- VALIDATING - EPOCH 178 -----
VAL loss: 0.01798 (epoch: 178, step: 0)
EPOCH IoU on VAL set:  53.18 %
----- TRAINING - EPOCH 179 -----
LEARNING RATE:  0.00016308846431783376
loss: 0.01853 (epoch: 179, step: 0)
loss: 0.01852 (epoch: 179, step: 50)
loss: 0.01932 (epoch: 179, step: 100)
----- VALIDATING - EPOCH 179 -----
VAL loss: 0.01631 (epoch: 179, step: 0)
EPOCH IoU on VAL set:  54.71 %
----- TRAINING - EPOCH 180 -----
LEARNING RATE:  0.00016104843555278344
loss: 0.01683 (epoch: 180, step: 0)
loss: 0.01858 (epoch: 180, step: 50)
loss: 0.01882 (epoch: 180, step: 100)
----- VALIDATING - EPOCH 180 -----
VAL loss: 0.01944 (epoch: 180, step: 0)
EPOCH IoU on VAL set:  53.72 %
----- TRAINING - EPOCH 181 -----
LEARNING RATE:  0.00015900553138349974
loss: 0.01813 (epoch: 181, step: 0)
loss: 0.01922 (epoch: 181, step: 50)
loss: 0.0194 (epoch: 181, step: 100)
----- VALIDATING - EPOCH 181 -----
VAL loss: 0.01626 (epoch

loss: 0.01719 (epoch: 206, step: 0)
loss: 0.01707 (epoch: 206, step: 50)
loss: 0.01713 (epoch: 206, step: 100)
----- VALIDATING - EPOCH 206 -----
VAL loss: 0.01396 (epoch: 206, step: 0)
EPOCH IoU on VAL set:  56.36 %
----- TRAINING - EPOCH 207 -----
LEARNING RATE:  0.00010469631724186405
loss: 0.01951 (epoch: 207, step: 0)
loss: 0.01782 (epoch: 207, step: 50)
loss: 0.01747 (epoch: 207, step: 100)
----- VALIDATING - EPOCH 207 -----
VAL loss: 0.01379 (epoch: 207, step: 0)
EPOCH IoU on VAL set:  57.58 %
Saving model as best
save: ../save//model_encoder_best.pth (epoch: 207)
----- TRAINING - EPOCH 208 -----
LEARNING RATE:  0.00010255234759654667
loss: 0.01766 (epoch: 208, step: 0)
loss: 0.01684 (epoch: 208, step: 50)
loss: 0.01685 (epoch: 208, step: 100)
----- VALIDATING - EPOCH 208 -----
VAL loss: 0.01476 (epoch: 208, step: 0)
EPOCH IoU on VAL set:  56.94 %
----- TRAINING - EPOCH 209 -----
LEARNING RATE:  0.0001004033857000599
loss: 0.02391 (epoch: 209, step: 0)
loss: 0.0168 (epoch: 209, 

----- VALIDATING - EPOCH 231 -----
VAL loss: 0.01141 (epoch: 231, step: 0)
EPOCH IoU on VAL set:  60.72 %
Saving model as best
save: ../save//model_encoder_best.pth (epoch: 231)
----- TRAINING - EPOCH 232 -----
LEARNING RATE:  4.9170229731239706e-05
loss: 0.01211 (epoch: 232, step: 0)
loss: 0.01512 (epoch: 232, step: 50)
loss: 0.01518 (epoch: 232, step: 100)
----- VALIDATING - EPOCH 232 -----
VAL loss: 0.01173 (epoch: 232, step: 0)
EPOCH IoU on VAL set:  60.53 %
----- TRAINING - EPOCH 233 -----
LEARNING RATE:  4.683486267022203e-05
loss: 0.01579 (epoch: 233, step: 0)
loss: 0.01517 (epoch: 233, step: 50)
loss: 0.01508 (epoch: 233, step: 100)
----- VALIDATING - EPOCH 233 -----
VAL loss: 0.01101 (epoch: 233, step: 0)
EPOCH IoU on VAL set:  61.20 %
Saving model as best
save: ../save//model_encoder_best.pth (epoch: 233)
----- TRAINING - EPOCH 234 -----
LEARNING RATE:  4.448647818548563e-05
loss: 0.01469 (epoch: 234, step: 0)
loss: 0.01469 (epoch: 234, step: 50)
loss: 0.01513 (epoch: 234, st

loss: 0.039 (epoch: 13, step: 0)
loss: 0.03415 (epoch: 13, step: 50)
loss: 0.03401 (epoch: 13, step: 100)
----- VALIDATING - EPOCH 13 -----
VAL loss: 0.03109 (epoch: 13, step: 0)
EPOCH IoU on VAL set:  34.28 %
----- TRAINING - EPOCH 14 -----
LEARNING RATE:  0.0004765379672568623
loss: 0.03217 (epoch: 14, step: 0)
loss: 0.03305 (epoch: 14, step: 50)
loss: 0.03258 (epoch: 14, step: 100)
----- VALIDATING - EPOCH 14 -----
VAL loss: 0.03121 (epoch: 14, step: 0)
EPOCH IoU on VAL set:  34.32 %
----- TRAINING - EPOCH 15 -----
LEARNING RATE:  0.0004747279470344123
loss: 0.04231 (epoch: 15, step: 0)
loss: 0.03188 (epoch: 15, step: 50)
loss: 0.03166 (epoch: 15, step: 100)
----- VALIDATING - EPOCH 15 -----
VAL loss: 0.02818 (epoch: 15, step: 0)
EPOCH IoU on VAL set:  35.24 %
----- TRAINING - EPOCH 16 -----
LEARNING RATE:  0.000472917159689316
loss: 0.03502 (epoch: 16, step: 0)
loss: 0.03028 (epoch: 16, step: 50)
loss: 0.03069 (epoch: 16, step: 100)
----- VALIDATING - EPOCH 16 -----
VAL loss: 0.030

...

loss: 0.01642 (epoch: 121, step: 0)
loss: 0.02 (epoch: 121, step: 50)
loss: 0.02043 (epoch: 121, step: 100)
----- VALIDATING - EPOCH 121 -----
VAL loss: 0.01832 (epoch: 121, step: 0)
EPOCH IoU on VAL set:  50.52 %
----- TRAINING - EPOCH 122 -----
LEARNING RATE:  0.0002756479302169693
loss: 0.02504 (epoch: 122, step: 0)
loss: 0.02 (epoch: 122, step: 50)
loss: 0.02058 (epoch: 122, step: 100)
----- VALIDATING - EPOCH 122 -----
VAL loss: 0.01905 (epoch: 122, step: 0)
EPOCH IoU on VAL set:  49.66 %
----- TRAINING - EPOCH 123 -----
LEARNING RATE:  0.00027372405759774416
loss: 0.01817 (epoch: 123, step: 0)
loss: 0.0203 (epoch: 123, step: 50)
loss: 0.02034 (epoch: 123, step: 100)
----- VALIDATING - EPOCH 123 -----
VAL loss: 0.0185 (epoch: 123, step: 0)
EPOCH IoU on VAL set:  53.10 %
Saving model as best
save: ../save//model_best.pth (epoch: 123)
----- TRAINING - EPOCH 124 -----
LEARNING RATE:  0.00027179868134969597
loss: 0.02502 (epoch: 124, step: 0)
loss: 0.01993 (epoch: 124, step: 50)
loss:

EPOCH IoU on VAL set:  54.29 %
----- TRAINING - EPOCH 148 -----
LEARNING RATE:  0.00022510103857781495
loss: 0.01835 (epoch: 148, step: 0)
loss: 0.01876 (epoch: 148, step: 50)
loss: 0.01899 (epoch: 148, step: 100)
----- VALIDATING - EPOCH 148 -----
VAL loss: 0.01887 (epoch: 148, step: 0)
EPOCH IoU on VAL set:  53.29 %
----- TRAINING - EPOCH 149 -----
LEARNING RATE:  0.0002231331780756292
loss: 0.01648 (epoch: 149, step: 0)
loss: 0.01956 (epoch: 149, step: 50)
loss: 0.019 (epoch: 149, step: 100)
----- VALIDATING - EPOCH 149 -----
VAL loss: 0.0163 (epoch: 149, step: 0)
EPOCH IoU on VAL set:  53.98 %
----- TRAINING - EPOCH 150 -----
LEARNING RATE:  0.00022116338731992472
loss: 0.01696 (epoch: 150, step: 0)
loss: 0.01907 (epoch: 150, step: 50)
loss: 0.01922 (epoch: 150, step: 100)
----- VALIDATING - EPOCH 150 -----
VAL loss: 0.01568 (epoch: 150, step: 0)
EPOCH IoU on VAL set:  55.03 %
Saving model as best
save: ../save//model_best.pth (epoch: 150)
----- TRAINING - EPOCH 151 -----
LEARNING 

loss: 0.01657 (epoch: 174, step: 0)
loss: 0.01762 (epoch: 174, step: 50)
loss: 0.01792 (epoch: 174, step: 100)
----- VALIDATING - EPOCH 174 -----
VAL loss: 0.01391 (epoch: 174, step: 0)
EPOCH IoU on VAL set:  57.29 %
Saving model as best
save: ../save//model_best.pth (epoch: 174)
----- TRAINING - EPOCH 175 -----
LEARNING RATE:  0.00017122068475972232
loss: 0.01654 (epoch: 175, step: 0)
loss: 0.01769 (epoch: 175, step: 50)
loss: 0.01755 (epoch: 175, step: 100)
----- VALIDATING - EPOCH 175 -----
VAL loss: 0.01462 (epoch: 175, step: 0)
EPOCH IoU on VAL set:  56.26 %
----- TRAINING - EPOCH 176 -----
LEARNING RATE:  0.00016919173095082495
loss: 0.01962 (epoch: 176, step: 0)
loss: 0.01757 (epoch: 176, step: 50)
loss: 0.0176 (epoch: 176, step: 100)
----- VALIDATING - EPOCH 176 -----
VAL loss: 0.01394 (epoch: 176, step: 0)
EPOCH IoU on VAL set:  57.02 %
----- TRAINING - EPOCH 177 -----
LEARNING RATE:  0.00016716006998158345
loss: 0.01775 (epoch: 177, step: 0)
loss: 0.01761 (epoch: 177, step: 5

----- VALIDATING - EPOCH 200 -----
VAL loss: 0.01345 (epoch: 200, step: 0)
EPOCH IoU on VAL set:  58.20 %
----- TRAINING - EPOCH 201 -----
LEARNING RATE:  0.00011746189430880188
loss: 0.0172 (epoch: 201, step: 0)
loss: 0.01591 (epoch: 201, step: 50)
loss: 0.01603 (epoch: 201, step: 100)
----- VALIDATING - EPOCH 201 -----
VAL loss: 0.0131 (epoch: 201, step: 0)
EPOCH IoU on VAL set:  58.90 %
----- TRAINING - EPOCH 202 -----
LEARNING RATE:  0.0001153454502273269
loss: 0.01424 (epoch: 202, step: 0)
loss: 0.01566 (epoch: 202, step: 50)
loss: 0.01625 (epoch: 202, step: 100)
----- VALIDATING - EPOCH 202 -----
VAL loss: 0.01335 (epoch: 202, step: 0)
EPOCH IoU on VAL set:  59.20 %
Saving model as best
save: ../save//model_best.pth (epoch: 202)
----- TRAINING - EPOCH 203 -----
LEARNING RATE:  0.00011322468214628069
loss: 0.01789 (epoch: 203, step: 0)
loss: 0.01645 (epoch: 203, step: 50)
loss: 0.01614 (epoch: 203, step: 100)
----- VALIDATING - EPOCH 203 -----
VAL loss: 0.01405 (epoch: 203, step: 

EPOCH IoU on VAL set:  60.90 %
----- TRAINING - EPOCH 226 -----
LEARNING RATE:  6.294627058970836e-05
loss: 0.01329 (epoch: 226, step: 0)
loss: 0.01527 (epoch: 226, step: 50)
loss: 0.01494 (epoch: 226, step: 100)
----- VALIDATING - EPOCH 226 -----
VAL loss: 0.01166 (epoch: 226, step: 0)
EPOCH IoU on VAL set:  60.90 %
----- TRAINING - EPOCH 227 -----
LEARNING RATE:  6.0675604814245233e-05
loss: 0.01196 (epoch: 227, step: 0)
loss: 0.01444 (epoch: 227, step: 50)
loss: 0.0147 (epoch: 227, step: 100)
----- VALIDATING - EPOCH 227 -----
VAL loss: 0.01091 (epoch: 227, step: 0)
EPOCH IoU on VAL set:  60.64 %
----- TRAINING - EPOCH 228 -----
LEARNING RATE:  5.839545530477977e-05
loss: 0.0156 (epoch: 228, step: 0)
loss: 0.01479 (epoch: 228, step: 50)
loss: 0.01467 (epoch: 228, step: 100)
----- VALIDATING - EPOCH 228 -----
VAL loss: 0.01214 (epoch: 228, step: 0)
EPOCH IoU on VAL set:  60.93 %
----- TRAINING - EPOCH 229 -----
LEARNING RATE:  5.610536722255538e-05
loss: 0.01977 (epoch: 229, step: 0)

In [12]:
main(modell="erfnet")

========== ENCODER TRAINING ===========
<class '__main__.CrossEntropyLoss2d'>
----- TRAINING - EPOCH 1 -----
LEARNING RATE:  0.0005
loss: 3.542 (epoch: 1, step: 0)
loss: 0.4227 (epoch: 1, step: 50)
loss: 0.2555 (epoch: 1, step: 100)
----- VALIDATING - EPOCH 1 -----
VAL loss: 0.07267 (epoch: 1, step: 0)
EPOCH IoU on VAL set:  20.91 %
Saving model as best
save: ../save//model_encoder_best.pth (epoch: 1)
----- TRAINING - EPOCH 2 -----
LEARNING RATE:  0.000492493711467861
loss: 0.07811 (epoch: 2, step: 0)
loss: 0.06338 (epoch: 2, step: 50)
loss: 0.06228 (epoch: 2, step: 100)
----- VALIDATING - EPOCH 2 -----
VAL loss: 0.05968 (epoch: 2, step: 0)
EPOCH IoU on VAL set:  29.37 %
Saving model as best
save: ../save//model_encoder_best.pth (epoch: 2)
----- TRAINING - EPOCH 3 -----
LEARNING RATE:  0.0004849746889841331
loss: 0.04694 (epoch: 3, step: 0)
loss: 0.05442 (epoch: 3, step: 50)
loss: 0.05304 (epoch: 3, step: 100)
----- VALIDATING - EPOCH 3 -----
VAL loss: 0.05599 (epoch: 3, step: 0)
EPOCH

loss: 0.02723 (epoch: 26, step: 50)
loss: 0.02796 (epoch: 26, step: 100)
----- VALIDATING - EPOCH 26 -----
VAL loss: 0.02866 (epoch: 26, step: 0)
EPOCH IoU on VAL set:  42.52 %
----- TRAINING - EPOCH 27 -----
LEARNING RATE:  0.0002998920164149494
loss: 0.03232 (epoch: 27, step: 0)
loss: 0.02843 (epoch: 27, step: 50)
loss: 0.02834 (epoch: 27, step: 100)
----- VALIDATING - EPOCH 27 -----
VAL loss: 0.03811 (epoch: 27, step: 0)
EPOCH IoU on VAL set:  39.45 %
----- TRAINING - EPOCH 28 -----
LEARNING RATE:  0.00029194189645999016
loss: 0.02364 (epoch: 28, step: 0)
loss: 0.02798 (epoch: 28, step: 50)
loss: 0.02818 (epoch: 28, step: 100)
----- VALIDATING - EPOCH 28 -----
VAL loss: 0.03012 (epoch: 28, step: 0)
EPOCH IoU on VAL set:  44.65 %
Saving model as best
save: ../save//model_encoder_best.pth (epoch: 28)
----- TRAINING - EPOCH 29 -----
LEARNING RATE:  0.00028396764480896164
loss: 0.02369 (epoch: 29, step: 0)
loss: 0.02808 (epoch: 29, step: 50)
loss: 0.02833 (epoch: 29, step: 100)
----- VA

loss: 0.02067 (epoch: 53, step: 0)
loss: 0.02275 (epoch: 53, step: 50)
loss: 0.02293 (epoch: 53, step: 100)
----- VALIDATING - EPOCH 53 -----
VAL loss: 0.02251 (epoch: 53, step: 0)
EPOCH IoU on VAL set:  48.30 %
----- TRAINING - EPOCH 54 -----
LEARNING RATE:  7.231395505915424e-05
loss: 0.02224 (epoch: 54, step: 0)
loss: 0.02238 (epoch: 54, step: 50)
loss: 0.02222 (epoch: 54, step: 100)
----- VALIDATING - EPOCH 54 -----
VAL loss: 0.022 (epoch: 54, step: 0)
EPOCH IoU on VAL set:  49.38 %
Saving model as best
save: ../save//model_encoder_best.pth (epoch: 54)
----- TRAINING - EPOCH 55 -----
LEARNING RATE:  6.294627058970836e-05
loss: 0.02187 (epoch: 55, step: 0)
loss: 0.02182 (epoch: 55, step: 50)
loss: 0.02191 (epoch: 55, step: 100)
----- VALIDATING - EPOCH 55 -----
VAL loss: 0.02131 (epoch: 55, step: 0)
EPOCH IoU on VAL set:  48.21 %
----- TRAINING - EPOCH 56 -----
LEARNING RATE:  5.3420368916117334e-05
loss: 0.02655 (epoch: 56, step: 0)
loss: 0.02132 (epoch: 56, step: 50)
loss: 0.02187

loss: 0.04033 (epoch: 19, step: 0)
loss: 0.03613 (epoch: 19, step: 50)
loss: 0.03642 (epoch: 19, step: 100)
----- VALIDATING - EPOCH 19 -----
VAL loss: 0.03527 (epoch: 19, step: 0)
EPOCH IoU on VAL set:  30.77 %
Saving model as best
save: ../save//model_best.pth (epoch: 19)
----- TRAINING - EPOCH 20 -----
LEARNING RATE:  0.0003549272548125162
loss: 0.03257 (epoch: 20, step: 0)
loss: 0.03467 (epoch: 20, step: 50)
loss: 0.03522 (epoch: 20, step: 100)
----- VALIDATING - EPOCH 20 -----
VAL loss: 0.04313 (epoch: 20, step: 0)
EPOCH IoU on VAL set:  27.52 %
----- TRAINING - EPOCH 21 -----
LEARNING RATE:  0.00034712658133080355
loss: 0.0341 (epoch: 21, step: 0)
loss: 0.03525 (epoch: 21, step: 50)
loss: 0.03466 (epoch: 21, step: 100)
----- VALIDATING - EPOCH 21 -----
VAL loss: 0.03865 (epoch: 21, step: 0)
EPOCH IoU on VAL set:  33.23 %
Saving model as best
save: ../save//model_best.pth (epoch: 21)
----- TRAINING - EPOCH 22 -----
LEARNING RATE:  0.0003393063796290625
loss: 0.03425 (epoch: 22, st

loss: 0.0387 (epoch: 44, step: 0)
loss: 0.02517 (epoch: 44, step: 50)
loss: 0.02513 (epoch: 44, step: 100)
----- VALIDATING - EPOCH 44 -----
VAL loss: 0.02263 (epoch: 44, step: 0)
EPOCH IoU on VAL set:  46.30 %
Saving model as best
save: ../save//model_best.pth (epoch: 44)
----- TRAINING - EPOCH 45 -----
LEARNING RATE:  0.00015217449296258857
loss: 0.0303 (epoch: 45, step: 0)
loss: 0.02488 (epoch: 45, step: 50)
loss: 0.02468 (epoch: 45, step: 100)
----- VALIDATING - EPOCH 45 -----
VAL loss: 0.02387 (epoch: 45, step: 0)
EPOCH IoU on VAL set:  46.09 %
----- TRAINING - EPOCH 46 -----
LEARNING RATE:  0.00014358729437462936
loss: 0.02632 (epoch: 46, step: 0)
loss: 0.02516 (epoch: 46, step: 50)
loss: 0.02518 (epoch: 46, step: 100)
----- VALIDATING - EPOCH 46 -----
VAL loss: 0.02216 (epoch: 46, step: 0)
EPOCH IoU on VAL set:  47.25 %
Saving model as best
save: ../save//model_best.pth (epoch: 46)
----- TRAINING - EPOCH 47 -----
LEARNING RATE:  0.00013494261163740181
loss: 0.03807 (epoch: 47, s

In [13]:
main(modell="erfnet")

========== ENCODER TRAINING ===========
<class '__main__.CrossEntropyLoss2d'>
----- TRAINING - EPOCH 1 -----
LEARNING RATE:  0.0005
loss: 2.981 (epoch: 1, step: 0)
loss: 0.3336 (epoch: 1, step: 50)
loss: 0.207 (epoch: 1, step: 100)
loss: 0.162 (epoch: 1, step: 150)
loss: 0.1376 (epoch: 1, step: 200)
loss: 0.1215 (epoch: 1, step: 250)
loss: 0.1102 (epoch: 1, step: 300)
loss: 0.1022 (epoch: 1, step: 350)
loss: 0.09563 (epoch: 1, step: 400)
loss: 0.09041 (epoch: 1, step: 450)
loss: 0.08627 (epoch: 1, step: 500)
loss: 0.08262 (epoch: 1, step: 550)
loss: 0.07947 (epoch: 1, step: 600)
loss: 0.07681 (epoch: 1, step: 650)
loss: 0.07443 (epoch: 1, step: 700)
loss: 0.07239 (epoch: 1, step: 750)
loss: 0.07057 (epoch: 1, step: 800)
----- VALIDATING - EPOCH 1 -----
VAL loss: 0.05788 (epoch: 1, step: 0)
VAL loss: 0.05654 (epoch: 1, step: 50)
EPOCH IoU on VAL set:  25.36 %
Saving model as best
save: ../save//model_encoder_best.pth (epoch: 1)
----- TRAINING - EPOCH 2 -----
LEARNING RATE:  0.0004849746

loss: 0.02873 (epoch: 10, step: 400)
loss: 0.02872 (epoch: 10, step: 450)
loss: 0.0288 (epoch: 10, step: 500)
loss: 0.02875 (epoch: 10, step: 550)
loss: 0.02883 (epoch: 10, step: 600)
loss: 0.02883 (epoch: 10, step: 650)
loss: 0.02879 (epoch: 10, step: 700)
loss: 0.02894 (epoch: 10, step: 750)
loss: 0.02891 (epoch: 10, step: 800)
----- VALIDATING - EPOCH 10 -----
VAL loss: 0.03362 (epoch: 10, step: 0)
VAL loss: 0.03585 (epoch: 10, step: 50)
EPOCH IoU on VAL set:  39.11 %
----- TRAINING - EPOCH 11 -----
LEARNING RATE:  0.00034712658133080355
loss: 0.03045 (epoch: 11, step: 0)
loss: 0.02863 (epoch: 11, step: 50)
loss: 0.02888 (epoch: 11, step: 100)
loss: 0.0286 (epoch: 11, step: 150)
loss: 0.02844 (epoch: 11, step: 200)
loss: 0.02875 (epoch: 11, step: 250)
loss: 0.02868 (epoch: 11, step: 300)
loss: 0.02855 (epoch: 11, step: 350)
loss: 0.02871 (epoch: 11, step: 400)
loss: 0.02865 (epoch: 11, step: 450)
loss: 0.0286 (epoch: 11, step: 500)
loss: 0.02854 (epoch: 11, step: 550)
loss: 0.02857 

loss: 0.02681 (epoch: 19, step: 800)
----- VALIDATING - EPOCH 19 -----
VAL loss: 0.03588 (epoch: 19, step: 0)
VAL loss: 0.03468 (epoch: 19, step: 50)
EPOCH IoU on VAL set:  41.73 %
----- TRAINING - EPOCH 20 -----
LEARNING RATE:  0.0002026815849307756
loss: 0.02517 (epoch: 20, step: 0)
loss: 0.02561 (epoch: 20, step: 50)
loss: 0.02611 (epoch: 20, step: 100)
loss: 0.02665 (epoch: 20, step: 150)
loss: 0.02667 (epoch: 20, step: 200)
loss: 0.02665 (epoch: 20, step: 250)
loss: 0.02659 (epoch: 20, step: 300)
loss: 0.02655 (epoch: 20, step: 350)
loss: 0.02658 (epoch: 20, step: 400)
loss: 0.0266 (epoch: 20, step: 450)
loss: 0.02667 (epoch: 20, step: 500)
loss: 0.02673 (epoch: 20, step: 550)
loss: 0.02667 (epoch: 20, step: 600)
loss: 0.02672 (epoch: 20, step: 650)
loss: 0.02669 (epoch: 20, step: 700)
loss: 0.0267 (epoch: 20, step: 750)
loss: 0.02677 (epoch: 20, step: 800)
----- VALIDATING - EPOCH 20 -----
VAL loss: 0.03396 (epoch: 20, step: 0)
VAL loss: 0.03279 (epoch: 20, step: 50)
EPOCH IoU on

loss: 0.02518 (epoch: 29, step: 50)
loss: 0.02529 (epoch: 29, step: 100)
loss: 0.02525 (epoch: 29, step: 150)
loss: 0.0252 (epoch: 29, step: 200)
loss: 0.02529 (epoch: 29, step: 250)
loss: 0.02523 (epoch: 29, step: 300)
loss: 0.02525 (epoch: 29, step: 350)
loss: 0.02534 (epoch: 29, step: 400)
loss: 0.02539 (epoch: 29, step: 450)
loss: 0.02534 (epoch: 29, step: 500)
loss: 0.02532 (epoch: 29, step: 550)
loss: 0.0253 (epoch: 29, step: 600)
loss: 0.02529 (epoch: 29, step: 650)
loss: 0.02527 (epoch: 29, step: 700)
loss: 0.0253 (epoch: 29, step: 750)
loss: 0.02532 (epoch: 29, step: 800)
----- VALIDATING - EPOCH 29 -----
VAL loss: 0.02934 (epoch: 29, step: 0)
VAL loss: 0.02964 (epoch: 29, step: 50)
EPOCH IoU on VAL set:  44.74 %
----- TRAINING - EPOCH 30 -----
LEARNING RATE:  2.3418597108060762e-05
loss: 0.02701 (epoch: 30, step: 0)
loss: 0.02538 (epoch: 30, step: 50)
loss: 0.02516 (epoch: 30, step: 100)
loss: 0.02525 (epoch: 30, step: 150)
loss: 0.02543 (epoch: 30, step: 200)
loss: 0.02538 (

loss: 0.02827 (epoch: 8, step: 700)
loss: 0.02827 (epoch: 8, step: 750)
loss: 0.02825 (epoch: 8, step: 800)
----- VALIDATING - EPOCH 8 -----
VAL loss: 0.0325 (epoch: 8, step: 0)
VAL loss: 0.03503 (epoch: 8, step: 50)
EPOCH IoU on VAL set:  36.47 %
----- TRAINING - EPOCH 9 -----
LEARNING RATE:  0.00037821721103476613
loss: 0.02614 (epoch: 9, step: 0)
loss: 0.02889 (epoch: 9, step: 50)
loss: 0.02902 (epoch: 9, step: 100)
loss: 0.02867 (epoch: 9, step: 150)
loss: 0.02837 (epoch: 9, step: 200)
loss: 0.0283 (epoch: 9, step: 250)
loss: 0.0282 (epoch: 9, step: 300)
loss: 0.02818 (epoch: 9, step: 350)
loss: 0.0281 (epoch: 9, step: 400)
loss: 0.02815 (epoch: 9, step: 450)
loss: 0.02804 (epoch: 9, step: 500)
loss: 0.02804 (epoch: 9, step: 550)
loss: 0.0279 (epoch: 9, step: 600)
loss: 0.02792 (epoch: 9, step: 650)
loss: 0.02792 (epoch: 9, step: 700)
loss: 0.02793 (epoch: 9, step: 750)
loss: 0.02788 (epoch: 9, step: 800)
----- VALIDATING - EPOCH 9 -----
VAL loss: 0.03531 (epoch: 9, step: 0)
VAL lo

loss: 0.02679 (epoch: 18, step: 150)
loss: 0.02653 (epoch: 18, step: 200)
loss: 0.02652 (epoch: 18, step: 250)
loss: 0.02632 (epoch: 18, step: 300)
loss: 0.02641 (epoch: 18, step: 350)
loss: 0.02649 (epoch: 18, step: 400)
loss: 0.02654 (epoch: 18, step: 450)
loss: 0.02656 (epoch: 18, step: 500)
loss: 0.02657 (epoch: 18, step: 550)
loss: 0.02658 (epoch: 18, step: 600)
loss: 0.02667 (epoch: 18, step: 650)
loss: 0.0267 (epoch: 18, step: 700)
loss: 0.02677 (epoch: 18, step: 750)
loss: 0.02677 (epoch: 18, step: 800)
----- VALIDATING - EPOCH 18 -----
VAL loss: 0.02935 (epoch: 18, step: 0)
VAL loss: 0.03088 (epoch: 18, step: 50)
EPOCH IoU on VAL set:  40.96 %
----- TRAINING - EPOCH 19 -----
LEARNING RATE:  0.00021919164527704348
loss: 0.02613 (epoch: 19, step: 0)
loss: 0.02757 (epoch: 19, step: 50)
loss: 0.02696 (epoch: 19, step: 100)
loss: 0.02678 (epoch: 19, step: 150)
loss: 0.02672 (epoch: 19, step: 200)
loss: 0.0268 (epoch: 19, step: 250)
loss: 0.02679 (epoch: 19, step: 300)
loss: 0.02675

loss: 0.02539 (epoch: 27, step: 600)
loss: 0.02533 (epoch: 27, step: 650)
loss: 0.02535 (epoch: 27, step: 700)
loss: 0.0254 (epoch: 27, step: 750)
loss: 0.02536 (epoch: 27, step: 800)
----- VALIDATING - EPOCH 27 -----
VAL loss: 0.02696 (epoch: 27, step: 0)
VAL loss: 0.02772 (epoch: 27, step: 50)
EPOCH IoU on VAL set:  44.46 %
----- TRAINING - EPOCH 28 -----
LEARNING RATE:  6.294627058970836e-05
loss: 0.02368 (epoch: 28, step: 0)
loss: 0.0255 (epoch: 28, step: 50)
loss: 0.02541 (epoch: 28, step: 100)
loss: 0.02554 (epoch: 28, step: 150)
loss: 0.0255 (epoch: 28, step: 200)
loss: 0.02538 (epoch: 28, step: 250)
loss: 0.02545 (epoch: 28, step: 300)
loss: 0.0255 (epoch: 28, step: 350)
loss: 0.02539 (epoch: 28, step: 400)
loss: 0.02536 (epoch: 28, step: 450)
loss: 0.02532 (epoch: 28, step: 500)
loss: 0.02536 (epoch: 28, step: 550)
loss: 0.02534 (epoch: 28, step: 600)
loss: 0.02536 (epoch: 28, step: 650)
loss: 0.02534 (epoch: 28, step: 700)
loss: 0.02534 (epoch: 28, step: 750)
loss: 0.02535 (e

In [15]:
main(modell="erfnet", state=True)

========== ENCODER TRAINING ===========
<class '__main__.CrossEntropyLoss2d'>
----- TRAINING - EPOCH 1 -----
LEARNING RATE:  0.0005
loss: 1.609 (epoch: 1, step: 0)
loss: 0.2681 (epoch: 1, step: 50)
loss: 0.1679 (epoch: 1, step: 100)
loss: 0.132 (epoch: 1, step: 150)


KeyboardInterrupt: 

In [12]:
#main(modell="erfnet") 

========== ENCODER TRAINING ===========
<class '__main__.CrossEntropyLoss2d'>
----- TRAINING - EPOCH 1 -----
LEARNING RATE:  0.0005
loss: 1.392 (epoch: 1, step: 0)
loss: 0.1645 (epoch: 1, step: 50)
loss: 0.1174 (epoch: 1, step: 100)
loss: 0.0984 (epoch: 1, step: 150)
loss: 0.08735 (epoch: 1, step: 200)
loss: 0.07982 (epoch: 1, step: 250)
loss: 0.07422 (epoch: 1, step: 300)
loss: 0.07026 (epoch: 1, step: 350)
loss: 0.06684 (epoch: 1, step: 400)
loss: 0.06414 (epoch: 1, step: 450)
loss: 0.06179 (epoch: 1, step: 500)
loss: 0.05983 (epoch: 1, step: 550)
loss: 0.05805 (epoch: 1, step: 600)
loss: 0.05651 (epoch: 1, step: 650)
loss: 0.05523 (epoch: 1, step: 700)
----- VALIDATING - EPOCH 1 -----
VAL loss: 0.08014 (epoch: 1, step: 0)
VAL loss: 0.07924 (epoch: 1, step: 50)
EPOCH IoU on VAL set:  25.75 %
Saving model as best
save: ../save//model_encoder_best.pth (epoch: 1)
----- TRAINING - EPOCH 2 -----
LEARNING RATE:  0.00047744269081074987
loss: 0.0411 (epoch: 2, step: 0)
loss: 0.03519 (epoch: 

loss: 0.02722 (epoch: 11, step: 400)
loss: 0.02721 (epoch: 11, step: 450)
loss: 0.02725 (epoch: 11, step: 500)
loss: 0.02741 (epoch: 11, step: 550)
loss: 0.0274 (epoch: 11, step: 600)
loss: 0.02739 (epoch: 11, step: 650)
loss: 0.02741 (epoch: 11, step: 700)
----- VALIDATING - EPOCH 11 -----
VAL loss: 0.08868 (epoch: 11, step: 0)
VAL loss: 0.08808 (epoch: 11, step: 50)
EPOCH IoU on VAL set:  21.07 %
----- TRAINING - EPOCH 12 -----
LEARNING RATE:  0.00024370321958949772
loss: 0.02398 (epoch: 12, step: 0)
loss: 0.02636 (epoch: 12, step: 50)
loss: 0.02645 (epoch: 12, step: 100)
loss: 0.02684 (epoch: 12, step: 150)
loss: 0.02676 (epoch: 12, step: 200)
loss: 0.02697 (epoch: 12, step: 250)
loss: 0.02697 (epoch: 12, step: 300)
loss: 0.02701 (epoch: 12, step: 350)
loss: 0.02706 (epoch: 12, step: 400)
loss: 0.0271 (epoch: 12, step: 450)
loss: 0.02708 (epoch: 12, step: 500)
loss: 0.02703 (epoch: 12, step: 550)
loss: 0.02709 (epoch: 12, step: 600)
loss: 0.02712 (epoch: 12, step: 650)
loss: 0.02714

loss: 0.05171 (epoch: 2, step: 0)
loss: 0.0506 (epoch: 2, step: 50)
loss: 0.04893 (epoch: 2, step: 100)
loss: 0.04844 (epoch: 2, step: 150)
loss: 0.04786 (epoch: 2, step: 200)
loss: 0.04694 (epoch: 2, step: 250)
loss: 0.04642 (epoch: 2, step: 300)
loss: 0.04589 (epoch: 2, step: 350)
loss: 0.04523 (epoch: 2, step: 400)
loss: 0.04468 (epoch: 2, step: 450)
loss: 0.04428 (epoch: 2, step: 500)
loss: 0.04372 (epoch: 2, step: 550)
loss: 0.04325 (epoch: 2, step: 600)
loss: 0.04274 (epoch: 2, step: 650)
loss: 0.0423 (epoch: 2, step: 700)
----- VALIDATING - EPOCH 2 -----
VAL loss: 0.08127 (epoch: 2, step: 0)
VAL loss: 0.08087 (epoch: 2, step: 50)
EPOCH IoU on VAL set:  20.37 %
Saving model as best
save: ../save//model_best.pth (epoch: 2)
----- TRAINING - EPOCH 3 -----
LEARNING RATE:  0.00045476628804148113
loss: 0.03064 (epoch: 3, step: 0)
loss: 0.03612 (epoch: 3, step: 50)
loss: 0.03542 (epoch: 3, step: 100)
loss: 0.03497 (epoch: 3, step: 150)
loss: 0.03461 (epoch: 3, step: 200)
loss: 0.03494 (

loss: 0.02668 (epoch: 12, step: 600)
loss: 0.02668 (epoch: 12, step: 650)
loss: 0.02665 (epoch: 12, step: 700)
----- VALIDATING - EPOCH 12 -----
VAL loss: 0.09219 (epoch: 12, step: 0)
VAL loss: 0.09078 (epoch: 12, step: 50)
EPOCH IoU on VAL set:  24.60 %
----- TRAINING - EPOCH 13 -----
LEARNING RATE:  0.00021919164527704348
loss: 0.03037 (epoch: 13, step: 0)
loss: 0.02642 (epoch: 13, step: 50)
loss: 0.0262 (epoch: 13, step: 100)
loss: 0.02636 (epoch: 13, step: 150)
loss: 0.02638 (epoch: 13, step: 200)
loss: 0.02665 (epoch: 13, step: 250)
loss: 0.02659 (epoch: 13, step: 300)
loss: 0.02655 (epoch: 13, step: 350)
loss: 0.02652 (epoch: 13, step: 400)
loss: 0.02658 (epoch: 13, step: 450)
loss: 0.02653 (epoch: 13, step: 500)
loss: 0.02646 (epoch: 13, step: 550)
loss: 0.02648 (epoch: 13, step: 600)
loss: 0.02651 (epoch: 13, step: 650)
loss: 0.02646 (epoch: 13, step: 700)
----- VALIDATING - EPOCH 13 -----
VAL loss: 0.1055 (epoch: 13, step: 0)
VAL loss: 0.1049 (epoch: 13, step: 50)
EPOCH IoU on